### Opening The File

In [1]:
words = open('names.txt','r').read().splitlines()

In [164]:
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

### String To Integer 

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items() }


### Analysing the trigrams

In [165]:
b = {}
for w in words:
    chs = ['.']+['.']+  list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):#zip function combines 2 or more iterables into a single iterable 
        trigram = (ch1,ch2,ch3)
        b[trigram] = b.get(trigram,0)+1 #its a dictionary... b.get if trigram is there in dictionary it will incrrement by 1 else  initialised by 0
sorted(b.items(),key = lambda kv : -kv[1])[:5]

[(('.', '.', 'a'), 4410),
 (('.', '.', 'k'), 2963),
 (('.', '.', 'm'), 2538),
 (('.', '.', 'j'), 2422),
 (('.', '.', 's'), 2055)]

## Creating a training set

In [9]:
import torch
import torch.nn.functional as F

In [173]:
xs = []
ys = []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]): 
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
#         print(f'{ch1}{ch2}{ch3}')
        xs.append(ix1)
        xs.append(ix2)
        ys.append(ix3)
xs = torch.tensor(xs) # .tensor int64, Tensor float32, here we want counts so integers
ys = torch.tensor(ys)
print(xs.shape) # torch.Size([456292])
print(ys.shape) # torch.Size([228146])
num = ys.nelement()

print('number of examples: ', num)

g = torch.Generator().manual_seed(2147483647)
W1 = torch.randn((27*2,27),requires_grad = True)


torch.Size([392226])
torch.Size([196113])
number of examples:  196113


### Gradient Descent

In [174]:
for k in range(100):
  
  # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
    #below 2 lines are converting shape of xenc from torch.Size([456292,27]) to [228146,54]
    xenc = torch.cat([xenc[i:i+2] for i in range(0, xenc.shape[0], 2)], dim=0)
    xenc = xenc.view(num, -1) 
    logits = xenc @ W1 # predict log-counts
    
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean()+ 0.01*(W1**2).mean()
    if(k>95):
        print(F'EPOCH = {k+1}, Loss = {loss.item():.4f}')
  # backward pass
    W1.grad = None # set to zero the gradient
    loss.backward()
  
  # update
    W1.data += -50 * W1.grad 

EPOCH = 97, Loss = 2.2747
EPOCH = 98, Loss = 2.2744
EPOCH = 99, Loss = 2.2741
EPOCH = 100, Loss = 2.2738


### PREDICTIONS 

In [195]:
g = torch.Generator().manual_seed(2147483647)
import random
for i in range(5):
  
    out = []
    a = random.randint(1,27)
    ix = [0,a]
    ix = torch.tensor(ix)
    while True:
        xenc = F.one_hot(ix, num_classes=27).float()
        xenc = torch.cat([xenc[i:i+2] for i in range(0, xenc.shape[0], 2)], dim=0)    # reshape the resulting 2D array
        xenc = xenc.view(1, -1)
        logits = xenc @ W1 # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True) # probabilities for next character
        d= torch.multinomial(p, num_samples=1, replacement=True, generator=g).flatten()
        out.append(itos[d.item()])
        if d.item() == 0:
            break
        ix = torch.cat((ix,d))
        ix = ix[1:]
        if( (len(''.join(out))<3) or len(''.join(out))>7):
            continue
        else:
            print(''.join(out))

ori
avo
avob
avobo
avobor
avobori
ril
rily
rilyf
rilyfn
rilyfnz
rem
rema
remal
remala
remalae


In [84]:
p = torch.rand(3,generator=g)# random numbers between 0and 1
p = p/p.sum()
p

tensor([0.2957, 0.1791, 0.5252])

In [90]:
torch.multinomial(p,num_samples = 10,replacement=True,generator=g) 


tensor([2, 0, 1, 0, 0, 2, 2, 2, 2, 2])